In [1]:
import pandas as pd

In [2]:
shared_cpg = pd.read_csv("CpGs_at_least_shared_by_two.csv", header = None)

In [3]:
coordinates = []
with open('KIPAN.hm450.coords', 'r') as file:
    for line in file:
        # Split the line into individual coordinate values and convert them to floats
        a, b, c, d = line.strip().split()
        coordinates.append([a,b,c,d])

In [4]:
def find_gene_symbol_by_cg_value(cg_value):
    none = 'NA'
    for item in coordinates:
        if item[0] == cg_value:
            return item[1]
    return none

In [5]:
def tosymbol(cpglist):
    gene_symbols = []
    i = 0
    for cpg in cpglist:
        i+=1
        if i % 100 == 0:
            print(i)
        if find_gene_symbol_by_cg_value(cpg) == None or find_gene_symbol_by_cg_value(cpg) == 'NA':
            continue
        gene_symbols.append(find_gene_symbol_by_cg_value(cpg))
    print(len(gene_symbols))
    return gene_symbols

In [6]:
genes = tosymbol(list(shared_cpg[0]))

100
200
300
400
500
600
700
800
900
311


In [7]:
genes = pd.DataFrame(genes)

In [8]:
split_df = genes[0].str.split(';', expand=True).stack().reset_index(level=1, drop=True).rename(0)

# Drop the original column and merge with the new DataFrame
genes = genes.drop(0, axis=1).join(split_df)

# Reset index if needed
genes = genes.reset_index(drop=True)

In [9]:
genes.to_csv("genes_two_clocks.csv")

In [10]:
pip install gseapy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
# %matplotlib inline
# %config InlineBackend.figure_format='retina' # mac
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt

In [12]:
genes

,0
0,ODC1
1,SNORA80B
2,C20orf112
3,RNPEPL1
4,TEX264
...,...
354,PRKAA2
355,CASKIN2
356,PLEKHG5
357,UBE2V1


In [13]:
pip install gseapy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
# %matplotlib inline
# %config InlineBackend.figure_format='retina' # mac
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
gene_list = list(genes[0])

In [17]:
enr = gp.enrich(gene_list= gene_list,
                 gene_sets=[
  'GO_Cellular_Component_2023',
  'KEGG_2021_Human',
  'Metabolomics_Workbench_Metabolites_2022',
  'GO_Molecular_Function_2023',

                          ],
                cutoff = 1,
                )

In [28]:
from gseapy import barplot, dotplot
# categorical scatterplot
ax = dotplot(enr.results,
              column="P-value",
              x="Gene_set", # set x-axis, so you could do a multi-sample/library comparison
              title = "Intersection of 2 Clocks (900 CpG, 359 Genes)",
              xticklabels_rot=70, # rotate x-tick labels
              ofname = "2clocks_dot.png",
              size = 15,
              top_term=5,
              figsize = (4, 9),
              cutoff = 1,
             )

/home/leelab/irao/.local/lib/python3.10/site-packages/gseapy/plot.py:671: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df[self.colname].replace(
/home/leelab/irao/.local/lib/python3.10/site-packages/gseapy/plot.py:671: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[self.colname].replace(


In [29]:
results = enr.results
results = results.sort_values(by = "Adjusted P-value")

In [31]:
results

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
435,GO_Molecular_Function_2023,Sialyltransferase Activity (GO:0008373),4/20,0.000358,0.113063,0,0,14.186416,112.576999,ST8SIA2;ST3GAL4;ST3GAL1;ST6GALNAC4
397,Metabolomics_Workbench_Metabolites_2022,Acetic Acid,3/18,0.003567,0.135546,0,0,11.317003,63.782939,ACY1;AMDHD2;ASPA
181,KEGG_2021_Human,Neuroactive ligand-receptor interaction,15/341,0.001056,0.228099,0,0,2.654153,18.189582,CALCA;PTGER2;PMCH;HTR1D;GLP2R;POMC;PYY;P2RX6;M...
0,GO_Cellular_Component_2023,Endopeptidase Complex (GO:1905369),3/14,0.001676,0.303322,0,0,15.435420,98.654816,COLEC10;CRADD;HTRA2
398,Metabolomics_Workbench_Metabolites_2022,Dehydroepiandrosterone,2/12,0.017949,0.306357,0,0,11.287356,45.377471,HSD3B2;CYP2E1
...,...,...,...,...,...,...,...,...,...,...
394,KEGG_2021_Human,Ras signaling pathway,1/232,0.983753,0.992947,0,0,0.240874,0.003946,PLCE1
750,GO_Molecular_Function_2023,Zinc Ion Binding (GO:0008270),1/341,0.997692,0.997692,0,0,0.162734,0.000376,MEFV
749,GO_Molecular_Function_2023,Transition Metal Ion Binding (GO:0046914),2/456,0.997293,0.997692,0,0,0.243000,0.000659,MEFV;LOXL2
395,KEGG_2021_Human,MAPK signaling pathway,1/294,0.994640,0.999266,0,0,0.189298,0.001017,MAPK8IP2
